# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [13]:
# Standard Library
import os
from datetime import datetime
from typing import Dict, List, Literal

# Third-Party Libraries
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from tavily import TavilyClient

# Local Application / Internal Modules
from lib.agents import Agent
from lib.llm import LLM
from lib.messages import AIMessage, SystemMessage, ToolMessage, UserMessage, BaseMessage
from lib.tooling import tool, Tool
from lib.vector_db import VectorStoreManager
from lib.long_memory import LongTermMemory
from lib.short_memory import MemoryFragment

In [2]:
# Load environment variables and set defaults if not present
load_dotenv()
CHORMA_PATH = os.getenv("CHORMA_PATH", "./chroma_db")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [3]:
@tool
def retrieve_game(query: str, num_rel_result: int=5) -> List[Dict]:
    """
    Semantic search: Finds relevant games in the vector database based on the query.

    Args:
        query (str): A natural language question or keywords about the game industry.
        num_rel_result (int): Number of relevant results to return.

    Returns:
        List[Dict]: A list of game dictionaries, each containing:
            - Platform: (e.g., Game Boy, PlayStation 5, Xbox 360)
            - Name: The title of the game
            - YearOfRelease: Release year for that platform
            - Description: Additional contextual details
    """
    db = VectorStoreManager(os.getenv("OPENAI_API_KEY"), db_path=CHORMA_PATH)
    collection = db.get_store(
        name="udaplay"
)
    
    results = collection.query(
        query_texts=[query],
        n_results=num_rel_result
    )

    games = []
    
    # Iterate through documents and metadatas simultaneously
    for i, _ in enumerate(results['documents'][0]):
        # Extract metadata and merge with the document (description)
        game_info = results['metadatas'][0][i].copy()
        # game_info['Description'] = doc
        games.append(game_info)
        
    return games

#### Evaluate Retrieval Tool

In [4]:
class EvaluationReport(BaseModel):
    """Structured evaluation report for retrieval results"""
    useful: bool = Field(description="Whether the documents are useful to answer the question")
    description: str = Field(description="Detailed explanation of the evaluation result")

@tool
def evaluate_retrieval(question: str, retrieved_docs: List[Dict]) -> Dict:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question.
    
    Args:
        question (str): original question from user
        retrieved_docs (List[Dict]): retrieved documents most similar to the user query in the Vector Database
    
    Returns:
        Dict: result includes:
            - useful: whether the documents are useful to answer the question
            - description: description about the evaluation result
    """
    llm = LLM(model="gpt-4o-mini")
    
    # Format the retrieved documents for the prompt
    docs_text = "\n".join([
        f"- {doc.get('Name', 'Unknown')}: {doc.get('Description', doc)}"
        for doc in retrieved_docs
    ])
    
    # Create the evaluation prompt
    evaluation_prompt = f"""Your task is to evaluate if the documents are enough to respond to the query.
Give a detailed explanation, so it's possible to take an action to accept it or not.

User Query: {question}

Retrieved Documents:
{docs_text}

Evaluate whether these documents contain sufficient information to answer the user's question."""
    
    # Get LLM evaluation using structured output
    response = llm.invoke(
        input=evaluation_prompt,
        response_format=EvaluationReport
    )
    
    # Parse the structured response
    evaluation = response.model_validate_json(response.content) if isinstance(response.content, str) else response.content
    
    return {
        "useful": evaluation.useful,
        "description": evaluation.description
    }


#### Game Web Search Tool

In [5]:
@tool
def web_search(query: str, 
               search_depth: Literal["basic", "advanced", "fast", "ultra-fast"] = "advanced") -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic', 'advanced', 'fast', or 'ultra-fast' (default: advanced)
    """
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

### Agent

In [6]:
agent = Agent(
    model_name="gpt-4o-mini",
    tools=[retrieve_game, evaluate_retrieval, web_search],
    instructions=(
        "You are Udaplay, an AI assistant specialized in video games.\n"
        "When a user asks about a game, use the tools to retrieve information.\n"
        "If the retrieved documents are not sufficient, use the web search tool to find more information.\n"
        "Always provide helpful and accurate responses."
    )
)

In [7]:
queries = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X realeased for Playstation 5?"
]

In [8]:
runs = []
for query in queries:
    run = agent.invoke(
        query=query
    )
    runs.append(run)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [ ]:
run_1 = agent.invoke(
    query="what was my first query?"
)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [ ]:
messages run_1.get_final_state()["messages"]

'Your first query was about the release date of Pokémon Gold and Silver.'

In [10]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

In [11]:
runs = agent.get_session_runs("default")
for i, run_object in enumerate(runs, 1):
    print(f"\n# Run {i}", run_object.metadata)
    print("Messages:")
    print_messages(run_object.get_final_state()["messages"])


# Run 1 {'run_id': '8a71bbd0-9cd8-4bf7-a922-8f590cc36130', 'start_timestamp': '2026-01-16 15:07:19.594066', 'end_timestamp': '2026-01-16 15:07:24.670880', 'snapshot_counts': 5}
Messages:
 -> (role = system, content = You are Udaplay, an AI assistant specialized in video games.
When a user asks about a game, use the tools to retrieve information.
If the retrieved documents are not sufficient, use the web search tool to find more information.
Always provide helpful and accurate responses., tool_calls = None)
 -> (role = user, content = When Pokémon Gold and Silver was released?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageFunctionToolCall(id='call_QRXZxT67IRvxN713Sn9rvqWx', function=Function(arguments='{"query":"Pokémon Gold and Silver","num_rel_result":1}', name='retrieve_game'), type='function')])
 -> (role = tool, content = "[{'Description': 'Second-generation Pok\u00e9mon games introducing new regions, Pok\u00e9mon, and gameplay mecha

### (Optional) Advanced

In [ ]:
def build_memory_registration_tool(ltm:LongTermMemory, owner:str, namespace:str):
    """
    Create a tool for agents to register new memories.
    
    This factory function creates a tool that allows AI agents to store new
    information about users in the long-term memory system. The tool is
    pre-configured with specific owner and namespace parameters.
    
    Args:
        ltm (LongTermMemory): The memory system instance to use
        owner (str): User identifier for memory ownership
        namespace (str): Namespace for organizing memories
        
    Returns:
        Tool: A configured tool for memory registration
    """
    def _register(content:str):
        ltm.register(
            MemoryFragment(
                content=content, 
                owner=owner,
                namespace=namespace
            )
        )
        return "Saved new memory"

    return Tool(
        func=_register, 
        name="register_memory", 
        description=(
            "Register a new memory or preference about the user, " 
            "so it can be useful later as context.\n"
            "Args:\n"
            "    content: The information to save"
        )
    )

In [ ]:
def build_memory_search_tool(ltm:LongTermMemory, owner:str, namespace:str):
    """
    Create a tool for agents to search existing memories.
    
    This factory function creates a tool that allows AI agents to retrieve
    relevant memories from the long-term memory system based on semantic
    similarity to a search query.
    
    Args:
        ltm (LongTermMemory): The memory system instance to use
        owner (str): User identifier for memory ownership
        namespace (str): Namespace to search within
        
    Returns:
        Tool: A configured tool for memory search
    """
    def _search(content:str):
        result = ltm.search(
            query_text=content,
            owner=owner,
            namespace=namespace,
            limit=3,
        )
        return str(tuple(zip(result.fragments, result.metadata['distances'])))

    return Tool(
        func=_search, 
        name="search_memory", 
        description=(
            "Search for a stored memory or preference about the user, " 
            "so it's useful as a context.\n"
            "Args:\n"
            "    content: The information to look for"
        )
    )

In [15]:
db = VectorStoreManager(os.getenv("OPENAI_API_KEY"), db_path="./long_term_memory_db")
ltm = LongTermMemory(db)

In [9]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes